# Tutorial 5: Supported APIs

In [ ]:
import os; os.chdir("..")
import credential
import ponder.snowflake
import modin.pandas as pd
snowflake_con = ponder.snowflake.connect(user=credential.params["user"],password=credential.params["password"],account=credential.params["account"],role=credential.params["role"],database=credential.params["database"],schema=credential.params["schema"],warehouse=credential.params["warehouse"])
ponder.snowflake.init(snowflake_con,enable_ssl=True)

Ponder aims to be a drop-in replacement for pandas, we are working to support as much of the pandas API as possible, but it is possible that certain pandas APIs are not currently supported in Ponder. You can find a full list of pandas APIs we support [here](https://docs.ponder.io/overviewAPI/dataframes.html).

### What happens when an API is not supported in Ponder?

In the case where you are using a function that is not yet supported, you will get an `NotImplementedError`. 

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/ponder-org/ponder-datasets/main/tpch/orders.csv", header=0)
num_df = df.select_dtypes("number")

For example, here we are using the `.corr()` function to compute the correlation matrix. This is not currently implemented in Ponder, so running this will throw a `NotImplementedError`.

In [ ]:
num_df.corr()

If you are running into this error, please send us the specific dataset and APIs that you’re using to support@ponder.io, so that we can suggest a possible solution. 

Note that in such cases, there may be ways to rewrite the query in a different way that leverages the [APIs we support](https://docs.ponder.io/overviewAPI/dataframes.html). 

### Workaround: Defaulting to Pandas

Alternatively, you can access the underlying pandas dataframe via the `_to_pandas` helper method. Note that when you call `_to_pandas`, your entire table from your warehouse is now pull into memory since pandas operates in memory. Beware that this can incur a high I/O cost if you have a very large table, so please do so with care. In this case, we have a very small table, so it is ok to default to pandas as a workaround.

In [ ]:
num_df.shape

Our original dataframe is a Modin DataFrame:

In [ ]:
type(num_df)

`_to_pandas` returns a pandas DataFrame:

In [ ]:
type(num_df._to_pandas())

We can then default to using pandas's implementation for `.corr()` to get our result. 

In [ ]:
num_df._to_pandas().corr()